# Obtención detalla actividades (Web Scrapping)
(web Ayuntamiento de Madrid)

[Enlace portal web Ayuntamiento de Madrid Actividades Infantiles](https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Actividades-infantiles/?vgnextfmt=default&vgnextoid=fdc579db15034710VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD)

Voy a trabajar como ejemplo con la [actividad 'Taller de ajedrez'](https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Taller-de-ajedrez-avanzado-infantil-juvenil-Biblioteca-Maria-Lejarraga/?vgnextfmt=default&vgnextoid=991163008c7c5910VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD)

![imagen web Ayuntamiento detalle actividad](../img/AyuntMadrid_actividades_01.png)

## Extracción actividad ejemplo de página 1

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from datetime import date
import re

# URL de la actividad de ejemplo
url = "https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Actividades-infantiles/?vgnextfmt=default&vgnextoid=fdc579db15034710VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD&page=1"

response = requests.get(url)
if response.status_code == 200:
    print("Correcto: el servidor respondió con código 200.")
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Buscar el contenedor principal que contiene las actividades
    contenedor_principal = soup.find('ul', class_='events-results docs')
    
    if contenedor_principal:
        print("Contenedor principal encontrado.")
        
        # Buscar todas las actividades dentro del contenedor
        actividades = contenedor_principal.find_all('div', class_='event-info')
        
        print(f"Se encontraron {len(actividades)} actividades.")
        
        # Obtener la primera actividad para análisis detallado
        if actividades:
            primera_actividad = actividades[0]
            
            # Extraer el enlace de la primera actividad
            elemento_enlace = primera_actividad.find('a', class_='event-link')
            if elemento_enlace and 'href' in elemento_enlace.attrs:
                url_detalle = elemento_enlace['href']
                if not url_detalle.startswith('http'):
                    url_detalle = "https://www.madrid.es" + url_detalle
                
                print(f"Accediendo a la página de detalle: {url_detalle}")
                
                # Hacer una solicitud a la página de detalle
                response_detalle = requests.get(url_detalle)
                
                if response_detalle.status_code == 200:
                    soup_detalle = BeautifulSoup(response_detalle.text, 'html.parser')
                    
                    # Lista para almacenar los datos de la actividad
                    datos_actividad = {}
                    
                    # Extraer título
                    titulo_elem = soup_detalle.find('h3', class_='summary-title')
                    datos_actividad['título'] = titulo_elem.text.strip() if titulo_elem else "Sin título"
                    
                    # Extraer descripción
                    descripcion_elem = soup_detalle.find('div', class_='tiny-text')
                    if descripcion_elem and descripcion_elem.p:
                        datos_actividad['descripción'] = descripcion_elem.p.text.strip()
                    else:
                        datos_actividad['descripción'] = "Sin descripción"
                    
                    # Extraer edad, inscripción, periodicidad, día_días, horario
                    tiny_text = soup_detalle.find('div', class_='tiny-text')
                    if tiny_text:
                        contenido_texto = tiny_text.get_text()
                        
                        # Buscar edad
                        patron_edad = re.search(r'(\d+\s+a\s+\d+\s+años|de\s+\d+\s+a\s+\d+\s+años)', contenido_texto)
                        datos_actividad['edad'] = patron_edad.group(0) if patron_edad else "No especificada"
                        
                        # Buscar inscripción
                        patron_inscripcion = re.search(r'([^.]*inscripción[^.]*\.)', contenido_texto, re.IGNORECASE)
                        datos_actividad['inscripción'] = patron_inscripcion.group(0).strip() if patron_inscripcion else "No especificada"
                        
                        # Buscar periodicidad
                        patrones_periodicidad = ['diaria', 'semanal', 'quincenal', 'mensual']
                        datos_actividad['periodicidad'] = "No especificada"
                        for patron in patrones_periodicidad:
                            if re.search(patron, contenido_texto, re.IGNORECASE):
                                datos_actividad['periodicidad'] = patron
                                break
                        
                        # Buscar días
                        dias_semana = ['lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'domingo']
                        dias_encontrados = []
                        for dia in dias_semana:
                            if re.search(dia, contenido_texto, re.IGNORECASE):
                                dias_encontrados.append(dia)
                        datos_actividad['día_días'] = ", ".join(dias_encontrados) if dias_encontrados else "No especificado"
                        
                        # Buscar horario
                        patron_horario = re.search(r'(\d{1,2}[:.]\d{2}\s*a\s*\d{1,2}[:.]\d{2}\s*horas)', contenido_texto)
                        datos_actividad['horario'] = patron_horario.group(0) if patron_horario else "No especificado"
                    
                    # Extraer fecha
                    fecha_elem = soup_detalle.find('p', class_='text-date')
                    datos_actividad['fecha'] = fecha_elem.text.strip() if fecha_elem else "Sin fecha"
                    
                    # Extraer lugar_nombre
                    lugar_elem = soup_detalle.find('a', class_='url fn')
                    datos_actividad['lugar_nombre'] = lugar_elem.text.strip() if lugar_elem else "Sin lugar"
                    
                    # Extraer lugar_dirección
                    direccion_elem = soup_detalle.find('dl', class_='dl-horz adr')
                    if direccion_elem and direccion_elem.find('dd'):
                        direccion = direccion_elem.find('dd').text.strip()
                        direccion = re.sub(r'\s+', ' ', direccion)  # Eliminar espacios múltiples
                        datos_actividad['lugar_dirección'] = direccion
                    else:
                        datos_actividad['lugar_dirección'] = "Sin dirección"
                    
                    # Extraer precio y recomendación
                    actividades_info = soup_detalle.find('div', class_='actividades-info')
                    if actividades_info:
                        precio_elem = actividades_info.find('p', class_='gratuita')
                        datos_actividad['precio'] = precio_elem.text.strip() if precio_elem else "No especificado"
                        
                        recomendacion_elem = actividades_info.find('p', class_='ninos')
                        datos_actividad['recomendación'] = recomendacion_elem.text.strip() if recomendacion_elem else "No especificado"
                    else:
                        datos_actividad['precio'] = "No especificado"
                        datos_actividad['recomendación'] = "No especificado"
                    
                    # Extraer url_ampliar_info (buscar bajo el encabezado "Amplíe información")
                    amplia_info_header = soup_detalle.find('h4', class_='title8', string='Amplíe información')
                    if amplia_info_header and amplia_info_header.find_next('p') and amplia_info_header.find_next('p').find('a'):
                        url_ampliar = amplia_info_header.find_next('p').find('a')['href']
                        if not url_ampliar.startswith('http'):
                            url_ampliar = "https://www.madrid.es" + url_ampliar
                        datos_actividad['url_ampliar_info'] = url_ampliar
                    else:
                        datos_actividad['url_ampliar_info'] = "No disponible"
                        
                    # Imprimir los resultados
                    print("\nDatos obtenidos de la actividad:")
                    for clave, valor in datos_actividad.items():
                        print(f"{clave}: {valor}")
                    
                    # Crear DataFrame y guardar CSV
                    df = pd.DataFrame([datos_actividad])
                    df.to_csv(f'{directorio}actividad_ejemplo.csv', index=False, encoding='utf-8-sig')
                    print(f"\nDatos guardados correctamente en '{directorio}actividad_ejemplo.csv'")
                else:
                    print(f"Error al acceder a la página de detalle: código {response_detalle.status_code}")
            else:
                print("No se encontró el enlace a la página de detalle.")
        else:
            print("No se encontraron actividades.")
    else:
        print("No se encontró el contenedor principal de actividades.")
else:
    print(f"Error: el servidor respondió con código {response.status_code}.")

Correcto: el servidor respondió con código 200.
Contenedor principal encontrado.
Se encontraron 25 actividades.
Accediendo a la página de detalle: https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Taller-de-ajedrez-avanzado-infantil-juvenil-Biblioteca-Maria-Lejarraga/?vgnextfmt=default&vgnextoid=991163008c7c5910VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD

Datos obtenidos de la actividad:
título: Taller de ajedrez avanzado (infantil-juvenil) Biblioteca María Lejárraga
descripción: Taller avanzado para el perfeccionamiento de la práctica del ajedrez, deporte de reconocidos beneficios a nivel cognitivo e intelectual que mejora la memoria, la concentración y permite ejercitar ambos hemisferios cerebrales.
edad: de 6 a 17 años
inscripción: Para participar es impresindible realizar inscripción previa en este enlace.
periodicidad: semanal
día_días: miércoles
horario: 18:30 a 19:30 horas
fecha: Del miércoles 19 de marzo de 2025 al mi

In [3]:
df

,título,descripción,edad,inscripción,periodicidad,día_días,horario,fecha,lugar_nombre,lugar_dirección,url_ampliar_info,precio,recomendación
0,Taller de ajedrez avanzado (infantil-juvenil) ...,Taller avanzado para el perfeccionamiento de l...,de 6 a 17 años,Para participar es impresindible realizar insc...,semanal,miércoles,18:30 a 19:30 horas,Del miércoles 19 de marzo de 2025 al miércoles...,Biblioteca Pública Municipal María Lejárraga (...,"CALLE PRINCESA DE EBOLI, 29 MADRID",https://www.madrid.es/portales/munimadrid/es/I...,Gratuito,Recomendado para niñas y niños


## Extracción todas las actividades de una página

He comprobado la extracción para un elemento ejemplo. Voy a realizar un bucle para obtener dichos valores de cada una de las 25 actividades de esa pagina 1, y ponerlo en un dataframe para visualizarlo bien.

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from datetime import date
import re
import time

# URL de la actividad de ejemplo
url = "https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Actividades-infantiles/?vgnextfmt=default&vgnextoid=fdc579db15034710VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD&page=1"

# Crear directorio para guardar el CSV si no existe
directorio = '../data/raw/'

# Lista para almacenar los datos de todas las actividades
todas_actividades = []

response = requests.get(url)
if response.status_code == 200:
    print("Correcto: el servidor respondió con código 200.")
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Buscar el contenedor principal que contiene las actividades
    contenedor_principal = soup.find('ul', class_='events-results docs')
    
    if contenedor_principal:
        print("Contenedor principal encontrado.")
        
        # Buscar todas las actividades dentro del contenedor
        actividades = contenedor_principal.find_all('div', class_='event-info')
        
        print(f"Se encontraron {len(actividades)} actividades.")
        print("Comenzando extracción de datos...\n")
        
        # Iterar sobre todas las actividades encontradas
        for indice, actividad in enumerate(actividades, 1):
            print(f"Procesando actividad {indice} de {len(actividades)}...")
            
            # Extraer el enlace de la actividad
            elemento_enlace = actividad.find('a', class_='event-link')
            if elemento_enlace and 'href' in elemento_enlace.attrs:
                url_detalle = elemento_enlace['href']
                if not url_detalle.startswith('http'):
                    url_detalle = "https://www.madrid.es" + url_detalle
                
                print(f"  Accediendo a la página de detalle: {url_detalle}")
                
                # Hacer una solicitud a la página de detalle con un pequeño retraso para no sobrecargar el servidor
                time.sleep(3)  # Esperar 3 segundos entre solicitudes
                response_detalle = requests.get(url_detalle)
                
                if response_detalle.status_code == 200:
                    soup_detalle = BeautifulSoup(response_detalle.text, 'html.parser')
                    
                    # Lista para almacenar los datos de la actividad
                    datos_actividad = {}
                    
                    # Extraer título
                    titulo_elem = soup_detalle.find('h3', class_='summary-title')
                    datos_actividad['título'] = titulo_elem.text.strip() if titulo_elem else "Sin título"
                    
                    # Extraer descripción
                    descripcion_elem = soup_detalle.find('div', class_='tiny-text')
                    if descripcion_elem and descripcion_elem.p:
                        datos_actividad['descripción'] = descripcion_elem.p.text.strip()
                    else:
                        datos_actividad['descripción'] = "Sin descripción"
                    
                    # Extraer edad, inscripción, periodicidad, día_días, horario
                    tiny_text = soup_detalle.find('div', class_='tiny-text')
                    if tiny_text:
                        contenido_texto = tiny_text.get_text()
                        
                        # Buscar edad
                        patron_edad = re.search(r'(\d+\s+a\s+\d+\s+años|de\s+\d+\s+a\s+\d+\s+años)', contenido_texto)
                        datos_actividad['edad'] = patron_edad.group(0) if patron_edad else "No especificada"
                        
                        # Buscar inscripción
                        patron_inscripcion = re.search(r'([^.]*inscripción[^.]*\.)', contenido_texto, re.IGNORECASE)
                        datos_actividad['inscripción'] = patron_inscripcion.group(0).strip() if patron_inscripcion else "No especificada"
                        
                        # Buscar periodicidad
                        patrones_periodicidad = ['diaria', 'semanal', 'quincenal', 'mensual']
                        datos_actividad['periodicidad'] = "No especificada"
                        for patron in patrones_periodicidad:
                            if re.search(patron, contenido_texto, re.IGNORECASE):
                                datos_actividad['periodicidad'] = patron
                                break
                        
                        # Buscar días
                        dias_semana = ['lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'domingo']
                        dias_encontrados = []
                        for dia in dias_semana:
                            if re.search(dia, contenido_texto, re.IGNORECASE):
                                dias_encontrados.append(dia)
                        datos_actividad['día_días'] = ", ".join(dias_encontrados) if dias_encontrados else "No especificado"
                        
                        # Buscar horario
                        patron_horario = re.search(r'(\d{1,2}[:.]\d{2}\s*a\s*\d{1,2}[:.]\d{2}\s*horas)', contenido_texto)
                        datos_actividad['horario'] = patron_horario.group(0) if patron_horario else "No especificado"
                    
                    # Extraer fecha
                    fecha_elem = soup_detalle.find('p', class_='text-date')
                    datos_actividad['fecha'] = fecha_elem.text.strip() if fecha_elem else "Sin fecha"
                    
                    # Extraer lugar_nombre
                    lugar_elem = soup_detalle.find('a', class_='url fn')
                    datos_actividad['lugar_nombre'] = lugar_elem.text.strip() if lugar_elem else "Sin lugar"
                    
                    # Extraer lugar_dirección
                    direccion_elem = soup_detalle.find('dl', class_='dl-horz adr')
                    if direccion_elem and direccion_elem.find('dd'):
                        direccion = direccion_elem.find('dd').text.strip()
                        direccion = re.sub(r'\s+', ' ', direccion)  # Eliminar espacios múltiples
                        datos_actividad['lugar_dirección'] = direccion
                    else:
                        datos_actividad['lugar_dirección'] = "Sin dirección"
                    
                    # Extraer precio y recomendación
                    actividades_info = soup_detalle.find('div', class_='actividades-info')
                    if actividades_info:
                        precio_elem = actividades_info.find('p', class_='gratuita')
                        datos_actividad['precio'] = precio_elem.text.strip() if precio_elem else "No especificado"
                        
                        recomendacion_elem = actividades_info.find('p', class_='ninos')
                        datos_actividad['recomendación'] = recomendacion_elem.text.strip() if recomendacion_elem else "No especificado"
                    else:
                        datos_actividad['precio'] = "No especificado"
                        datos_actividad['recomendación'] = "No especificado"
                    
                    # Extraer url_ampliar_info (buscar bajo el encabezado "Amplíe información")
                    amplia_info_header = soup_detalle.find('h4', class_='title8', string='Amplíe información')
                    if amplia_info_header and amplia_info_header.find_next('p') and amplia_info_header.find_next('p').find('a'):
                        url_ampliar = amplia_info_header.find_next('p').find('a')['href']
                        if not url_ampliar.startswith('http'):
                            url_ampliar = "https://www.madrid.es" + url_ampliar
                        datos_actividad['url_ampliar_info'] = url_ampliar
                    else:
                        datos_actividad['url_ampliar_info'] = "No disponible"
                        
                    # Añadir URL de la página de detalle para referencia
                    datos_actividad['url_detalle'] = url_detalle
                        
                    # Mostrar información básica de la actividad extraída
                    print(f"  ✓ Extraída: {datos_actividad['título']}")
                    print(f"    Lugar: {datos_actividad['lugar_nombre']}")
                    print(f"    Edad: {datos_actividad['edad']}")
                    print(f"    Fecha: {datos_actividad['fecha']}")
                    print(f"    Precio: {datos_actividad['precio']}")
                    print()
                    
                    # Añadir los datos de esta actividad a la lista general
                    todas_actividades.append(datos_actividad)
                else:
                    print(f"  ✗ Error al acceder a la página de detalle: código {response_detalle.status_code}")
            else:
                print("  ✗ No se encontró el enlace a la página de detalle.")
        
        # Crear DataFrame con todas las actividades y guardar CSV
        if todas_actividades:
            df = pd.DataFrame(todas_actividades)
            nombre_archivo = f'{directorio}actividades_pagina1.csv'
            df.to_csv(nombre_archivo, index=False, encoding='utf-8-sig')
            print(f"\nDatos guardados correctamente en '{nombre_archivo}'")
            print(f"Se han extraído datos de {len(todas_actividades)} actividades.")
        else:
            print("No se pudieron extraer datos de ninguna actividad.")
    else:
        print("No se encontró el contenedor principal de actividades.")
else:
    print(f"Error: el servidor respondió con código {response.status_code}.")

Correcto: el servidor respondió con código 200.


Contenedor principal encontrado.
Se encontraron 25 actividades.
Comenzando extracción de datos...

Procesando actividad 1 de 25...
  Accediendo a la página de detalle: https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Taller-de-ajedrez-avanzado-infantil-juvenil-Biblioteca-Maria-Lejarraga/?vgnextfmt=default&vgnextoid=991163008c7c5910VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD
  ✓ Extraída: Taller de ajedrez avanzado (infantil-juvenil) Biblioteca María Lejárraga
    Lugar: Biblioteca Pública Municipal María Lejárraga (Hortaleza)
    Edad: de 6 a 17 años
    Fecha: Del miércoles 19 de marzo de 2025 al miércoles 4 de junio de 2025
    Precio: Gratuito

Procesando actividad 2 de 25...
  Accediendo a la página de detalle: https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Manualidades/?vgnextfmt=default&vgnextoid=3da790ec79b45910VgnVCM2000001f4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD


KeyboardInterrupt: 

In [ ]:
# visualización idéntica al ejemplo
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from datetime import date
import re
import time

# URL de la actividad de ejemplo
url = "https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Actividades-infantiles/?vgnextfmt=default&vgnextoid=fdc579db15034710VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD&page=1"

# Crear directorio para guardar el CSV si no existe
directorio = '../data/raw/' # Directorio de guardado de datos

# Lista para almacenar los datos de todas las actividades
todas_actividades = []

response = requests.get(url)
if response.status_code == 200:
    print("Correcto: el servidor respondió con código 200.")
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Buscar el contenedor principal que contiene las actividades
    contenedor_principal = soup.find('ul', class_='events-results docs')
    
    if contenedor_principal:
        print("Contenedor principal encontrado.")
        
        # Buscar todas las actividades dentro del contenedor
        actividades = contenedor_principal.find_all('div', class_='event-info')
        
        print(f"Se encontraron {len(actividades)} actividades.")
        
        # Iterar sobre todas las actividades encontradas
        for indice, actividad in enumerate(actividades, 1):
            print(f"\nProcesando actividad {indice} de {len(actividades)}...")
            
            # Extraer el enlace de la actividad
            elemento_enlace = actividad.find('a', class_='event-link')
            if elemento_enlace and 'href' in elemento_enlace.attrs:
                url_actividad = elemento_enlace['href']
                if not url_actividad.startswith('http'):
                    url_actividad = "https://www.madrid.es" + url_actividad
                
                print(f"Accediendo a la página de detalle: {url_actividad}")
                
                # Hacer una solicitud a la página de detalle con un pequeño retraso para no sobrecargar el servidor
                time.sleep(2)  # Esperar 2 segundos entre solicitudes
                response_detalle = requests.get(url_actividad)
                
                if response_detalle.status_code == 200:
                    soup_detalle = BeautifulSoup(response_detalle.text, 'html.parser')
                    
                    # Lista para almacenar los datos de la actividad
                    datos_actividad = {}
                    
                    # Extraer título
                    titulo_elem = soup_detalle.find('h3', class_='summary-title')
                    datos_actividad['título'] = titulo_elem.text.strip() if titulo_elem else "Sin título"
                    
                    # Extraer descripción
                    descripcion_elem = soup_detalle.find('div', class_='tiny-text')
                    if descripcion_elem and descripcion_elem.p:
                        datos_actividad['descripción'] = descripcion_elem.p.text.strip()
                    else:
                        datos_actividad['descripción'] = "Sin descripción"
                    
                    # Extraer edad, inscripción, periodicidad, día_días, horario
                    tiny_text = soup_detalle.find('div', class_='tiny-text')
                    if tiny_text:
                        contenido_texto = tiny_text.get_text()
                        
                        # Buscar edad
                        patron_edad = re.search(r'(\d+\s+a\s+\d+\s+años|de\s+\d+\s+a\s+\d+\s+años)', contenido_texto)
                        datos_actividad['edad'] = patron_edad.group(0) if patron_edad else "No especificada"
                        
                        # Buscar inscripción
                        patron_inscripcion = re.search(r'([^.]*inscripción[^.]*\.)', contenido_texto, re.IGNORECASE)
                        datos_actividad['inscripción'] = patron_inscripcion.group(0).strip() if patron_inscripcion else "No especificada"
                        
                        # Buscar periodicidad
                        patrones_periodicidad = ['diaria', 'semanal', 'quincenal', 'mensual']
                        datos_actividad['periodicidad'] = "No especificada"
                        for patron in patrones_periodicidad:
                            if re.search(patron, contenido_texto, re.IGNORECASE):
                                datos_actividad['periodicidad'] = patron
                                break
                        
                        # Buscar días
                        dias_semana = ['lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'domingo']
                        dias_encontrados = []
                        for dia in dias_semana:
                            if re.search(dia, contenido_texto, re.IGNORECASE):
                                dias_encontrados.append(dia)
                        datos_actividad['día_días'] = ", ".join(dias_encontrados) if dias_encontrados else "No especificado"
                        
                        # Buscar horario
                        patron_horario = re.search(r'(\d{1,2}[:.]\d{2}\s*a\s*\d{1,2}[:.]\d{2}\s*horas)', contenido_texto)
                        datos_actividad['horario'] = patron_horario.group(0) if patron_horario else "No especificado"
                    
                    # Extraer fecha
                    fecha_elem = soup_detalle.find('p', class_='text-date')
                    datos_actividad['fecha'] = fecha_elem.text.strip() if fecha_elem else "Sin fecha"
                    
                    # Extraer lugar_nombre
                    lugar_elem = soup_detalle.find('a', class_='url fn')
                    datos_actividad['lugar_nombre'] = lugar_elem.text.strip() if lugar_elem else "Sin lugar"
                    
                    # Extraer lugar_dirección
                    direccion_elem = soup_detalle.find('dl', class_='dl-horz adr')
                    if direccion_elem and direccion_elem.find('dd'):
                        direccion = direccion_elem.find('dd').text.strip()
                        direccion = re.sub(r'\s+', ' ', direccion)  # Eliminar espacios múltiples
                        datos_actividad['lugar_dirección'] = direccion
                    else:
                        datos_actividad['lugar_dirección'] = "Sin dirección"
                    
                    # Extraer precio y recomendación
                    actividades_info = soup_detalle.find('div', class_='actividades-info')
                    if actividades_info:
                        precio_elem = actividades_info.find('p', class_='gratuita')
                        datos_actividad['precio'] = precio_elem.text.strip() if precio_elem else "No especificado"
                        
                        recomendacion_elem = actividades_info.find('p', class_='ninos')
                        datos_actividad['recomendación'] = recomendacion_elem.text.strip() if recomendacion_elem else "No especificado"
                    else:
                        datos_actividad['precio'] = "No especificado"
                        datos_actividad['recomendación'] = "No especificado"
                    
                    # Extraer url_ampliar_info (buscar bajo el encabezado "Amplíe información")
                    amplia_info_header = soup_detalle.find('h4', class_='title8', string='Amplíe información')
                    if amplia_info_header and amplia_info_header.find_next('p') and amplia_info_header.find_next('p').find('a'):
                        url_ampliar = amplia_info_header.find_next('p').find('a')['href']
                        if not url_ampliar.startswith('http'):
                            url_ampliar = "https://www.madrid.es" + url_ampliar
                        datos_actividad['url_ampliar_info'] = url_ampliar
                    else:
                        datos_actividad['url_ampliar_info'] = "No disponible"
                        
                    # Añadir URL de la página de detalle para referencia
                    datos_actividad['url_actividad'] = url_actividad
                        
                    # Imprimir los resultados en el formato exacto solicitado
                    print("\nDatos obtenidos de la actividad:")
                    for clave, valor in datos_actividad.items():
                        if clave != 'url_actividad':  # No mostrar la url_actividad para mantener el formato original
                            print(f"{clave}: {valor}")
                    
                    # Añadir los datos de esta actividad a la lista general
                    todas_actividades.append(datos_actividad)
                else:
                    print(f"Error al acceder a la página de detalle: código {response_detalle.status_code}")
            else:
                print("No se encontró el enlace a la página de detalle.")
        
        # Crear DataFrame con todas las actividades y guardar CSV
        if todas_actividades:
            df = pd.DataFrame(todas_actividades)
            nombre_archivo = f'{directorio}actividades_pagina1.csv'
            df.to_csv(nombre_archivo, index=False, encoding='utf-8-sig')
            print(f"\nDatos guardados correctamente en '{nombre_archivo}'")
            print(f"Se han extraído datos de {len(todas_actividades)} actividades.")
        else:
            print("No se pudieron extraer datos de ninguna actividad.")
    else:
        print("No se encontró el contenedor principal de actividades.")
else:
    print(f"Error: el servidor respondió con código {response.status_code}.")

Correcto: el servidor respondió con código 200.
Contenedor principal encontrado.
Se encontraron 25 actividades.

Procesando actividad 1 de 25...
Accediendo a la página de detalle: https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Taller-de-ajedrez-avanzado-infantil-juvenil-Biblioteca-Maria-Lejarraga/?vgnextfmt=default&vgnextoid=991163008c7c5910VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD

Datos obtenidos de la actividad:
título: Taller de ajedrez avanzado (infantil-juvenil) Biblioteca María Lejárraga
descripción: Taller avanzado para el perfeccionamiento de la práctica del ajedrez, deporte de reconocidos beneficios a nivel cognitivo e intelectual que mejora la memoria, la concentración y permite ejercitar ambos hemisferios cerebrales.
edad: de 6 a 17 años
inscripción: Para participar es impresindible realizar inscripción previa en este enlace.
periodicidad: semanal
día_días: miércoles
horario: 18:30 a 19:30 horas
fecha: Del mi

In [7]:
df

,título,descripción,edad,inscripción,periodicidad,día_días,horario,fecha,lugar_nombre,lugar_dirección,precio,recomendación,url_ampliar_info,url_detalle
0,Taller de ajedrez avanzado (infantil-juvenil) ...,Taller avanzado para el perfeccionamiento de l...,de 6 a 17 años,Para participar es impresindible realizar insc...,semanal,miércoles,18:30 a 19:30 horas,Del miércoles 19 de marzo de 2025 al miércoles...,Biblioteca Pública Municipal María Lejárraga (...,"CALLE PRINCESA DE EBOLI, 29 MADRID",Gratuito,Recomendado para niñas y niños,https://www.madrid.es/portales/munimadrid/es/I...,https://www.madrid.es/portales/munimadrid/es/I...
1,Manualidades,"Taller de manualidades ideado, coordinado y ej...",de 6 a 12 años,No especificada,No especificada,martes,No especificado,El proceso de inscripción finaliza el 23/06/2025,Biblioteca Pública Municipal Gabriel García Má...,"PLAZA PUEBLO, 2 (CENTRO CULTURAL ORCASUR) 2804...",Gratuito,Recomendado para niñas y niños,No disponible,https://www.madrid.es/portales/munimadrid/es/I...
2,Talleres de Price (De 7 a 9 años),¡Diviértete aprendiendo mientras descubres tod...,7 a 9 años,No especificada,No especificada,No especificado,No especificado,Del sábado 29 de marzo de 2025 al sábado 14 de...,Teatro Circo Price,"RONDA ATOCHA, 35 28012 MADRID",No especificado,Recomendado para niñas y niños,No disponible,https://www.madrid.es/portales/munimadrid/es/I...
3,Talleres del Price (5 y 6 años),¡Diviértete aprendiendo mientras descubres tod...,No especificada,No especificada,No especificada,No especificado,No especificado,Del sábado 29 de marzo de 2025 al sábado 14 de...,Teatro Circo Price,"RONDA ATOCHA, 35 28012 MADRID",No especificado,No especificado,No disponible,https://www.madrid.es/portales/munimadrid/es/I...
4,LUDOTECAS,Sábados de abril y laborables no lectivos de S...,No especificada,"Actividad gratuita y de carácter abierto, sin ...",No especificada,sábado,10:30 a 13:30 horas,Del sábado 5 de abril de 2025 al sábado 26 de ...,Centro Sociocultural Valverde (Fuencarral - El...,"PLAZA ISLAS AZORES, 1 MADRID",Gratuito,Recomendado para niñas y niños,https://www.madrid.es/portales/munimadrid/es/I...,https://www.madrid.es/portales/munimadrid/es/I...
5,TALLERES INFANTILES,Sábados 5 y 19. MONSTRUOS DEL RIO: Para menore...,de 6 a 8 años,No especificada,No especificada,"sábado, domingo",No especificado,Del sábado 5 de abril de 2025 al domingo 27 de...,Centro de Interpretación de la Naturaleza Mont...,"Carretera M-607 Km 13 L-10, null MADRID",Gratuito,Recomendado para niñas y niños,https://www.madrid.es/portales/munimadrid/es/I...,https://www.madrid.es/portales/munimadrid/es/I...
6,Taller infantil. Reciclo y utilizo,Reciclado creativo para pequeños ecologistas: ...,de 6 a 12 años,No especificada,No especificada,sábado,No especificado,Del sábado 5 de abril de 2025 al sábado 26 de ...,Centro Cultural Carril del Conde (Hortaleza),"CALLE CARRIL DEL CONDE, 57 MADRID",Gratuito,Recomendado para niñas y niños,https://www.madrid.es/portales/munimadrid/es/I...,https://www.madrid.es/portales/munimadrid/es/I...
7,LUDOTECAS,Sábados de abril y días laborales no lectivos ...,No especificada,"Actividad gratuita y de carácter abierto, sin ...",No especificada,sábado,No especificado,Del sábado 5 de abril de 2025 al sábado 26 de ...,Centro Sociocultural Alfonso XII (Fuencarral -...,"CALLE MIRA EL RIO, 4 MADRID",Gratuito,Recomendado para niñas y niños,https://www.madrid.es/portales/munimadrid/es/I...,https://www.madrid.es/portales/munimadrid/es/I...
8,Deportes populares y tradicionales para familias,Para todos los públicos. Sin inscripción previ...,No especificada,Sin inscripción previa.,No especificada,sábado,No especificado,Del sábado 5 de abril de 2025 al sábado 26 de ...,Centro Cultural Galileo (Chamberí),"CALLE GALILEO, 39 MADRID",Gratuito,Recomendado para niñas y niños,https://www.madrid.es/portales/munimadrid/es/I...,https://www.madrid.es/portales/munimadrid/es/I...
9,TALLERES FAMILIARES,Para familias con menores de diferentes e